#### Installing the required packages

In [103]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### Importing libraries

In [185]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re

#### Defining the methods to crawlers the data

In [209]:
# Get content from the PPGTI website
def get_content():
    url = 'https://www.ifpb.edu.br/ppgti/programa/corpo-docente'
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    return soup

# Get the content from a tag
def get_tag_content(soup: bs, tag: str):
    return soup.find_all(tag,id=True)

# Get faculty table
def get_faculty(content):
    for elements in content:
        if "content" in elements.get('id'):
            if "content-core" in elements.get('id'):
                return elements
    return None

# Get the Teachers' name
def get_list_name(elements):
    h4_tags = []
    for el in elements.find_all('h4'):
        if len(el.text) != 0:
            h4_tags.append(el.text)
    return h4_tags

# Get the Research Approach
def get_list_research_approach(elements):
    list_research_approach = []
    research_pattern = r'Linha de Pesquisa:\s*(.*?)(?=Currículo Lattes|$)'
    for el in elements.find_all('p'):
        if len(el.text.strip()) != 0:
            research_match = re.search(research_pattern, el.text)
            evalute_string = research_match.group(1).strip() if research_match else None
            list_research_approach.append(evalute_string)
    return list_research_approach

# Get the Emails
def get_list_emails(elements):
    list_emails = []
    email_pattern = r'E-mail:\s*(\S+@\S+)'
    for el in elements.find_all('p'):
        if len(el.text.strip()) != 0:
            email_match = re.search(email_pattern, el.text)
            evalute_string = email_match.group(1) if email_match else None
            list_emails.append(evalute_string)
    return list_emails

# Get the Lattes url
def get_list_lattes_url(elements):
    list_url = []
    url_tags = elements.find_all('a')
    for link in url_tags:
        if 'mailto' not in link.get('href'):
            list_url.append(link.get('href'))
    return list_url


#### Getting the content of the page

In [130]:
soup = get_content()

#### Getting the content from a tag

In [131]:
content = get_tag_content(soup, 'div')

#### Getting the Faculty's table

In [141]:
elements = get_faculty(content)

#### Scraping and creating a dataframe to show all teachers

In [212]:
h4_tags = get_list_name(elements)
research_tags = get_list_research_approach(elements)
url_tags = get_list_lattes_url(elements)
email_tags = get_list_emails(elements)

list_data = []

# Testing whether lists have same length
if len(h4_tags) != len(research_tags) and len(h4_tags) != len(url_tags) and len(h4_tags) != len(email_tags):
    print("Error list does not match!")

for name, research, url, email in zip(h4_tags, research_tags, url_tags, email_tags):
    data = {'nome': name, 'linha_pesquisa': research, 'url_lattes': url, 'email': email}
    # print(f'Nome: {name} \n research: {research} url:{url} email: {email}')
    list_data.append(data)

df = pd.DataFrame(list_data)
df

,nome,linha_pesquisa,url_lattes,email
0,Dra. Damires Yluska de Souza Fernandes,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9411293540915384,damires@ifpb.edu.br
1,Dr. Diego Ernesto Rosa Pessoa,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/4442363726982518,diego.pessoa@ifpb.edu.br
2,Dr. Francisco Petrônio Alencar de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9716270626654261,petronio@ifpb.edu.br
3,Dra. Heremita Brasileiro Lira,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/3532701873325164,heremita@ifpb.edu.br
4,Dra. Juliana Dantas Ribeiro Viana de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9730254173461923,juliana.medeiros@ifpb.edu.br
5,Dr. Katyusco de Farias Santos,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/1246085373474860,katyusco.santos@ifpb.edu.br
6,Dr. Lafayette Batista Melo,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/2144574905285987,lafayette.melo@ifpb.edu.br
7,Dra. Luciana Pereira Oliveira,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/4044015211881197,luciana.oliveira@ifpb.edu.br
8,Dr. Paulo Ditarso Maciel Júnior,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/1101383196957378,paulo.maciel@ifpb.edu.br
9,Dr. Paulo Ribeiro Lins Júnior,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/2720405345746056,paulo.lins@ifpb.edu.br
